# Visualización: proyecto

## Conjuntos de datos

In [20]:
import pandas as pd
import geopandas as gpd
import numpy as np
from plotnine import *
import re

### `df_p_activa`

In [21]:
df_p_activa = pd.read_csv('../data/empleo/datasets-ISTAC_eparegActiva-1921.csv')

# Eliminamos las columnas sobrantes
df_p_activa.drop(
    columns = [
        'ESTADO_OBSERVACION#es',
        'CONFIDENCIALIDAD_OBSERVACION#es',
        'NOTAS_OBSERVACION#es'
    ],
    inplace = True
)

# Eliminamos las entradas con SEXO_CODE == _T (Total)
df_p_activa.drop(
    df_p_activa[df_p_activa['SEXO_CODE'] == '_T'].index,
    inplace = True
)

# Eliminamos las entradas de cada isla
rows_islas = df_p_activa[df_p_activa['TERRITORIO_CODE'].str.contains(r'ES\d{3}')].index
df_p_activa.drop(
    rows_islas,
    inplace = True
)

# Renombramos la columna 'OBS_VALUE' a 'FREQUENCY'
df_p_activa.rename(
    columns = {'OBS_VALUE': 'FREQUENCY'},
    inplace = True
)

display(df_p_activa)

,TIME_PERIOD#es,TIME_PERIOD_CODE,TERRITORIO#es,TERRITORIO_CODE,SEXO#es,SEXO_CODE,EDAD#es,EDAD_CODE,SITUACION_LABORAL_REGISTRADA#es,SITUACION_LABORAL_REGISTRADA_CODE,FREQUENCY
0,2021 Cuarto trimestre,2021-Q4,Artenara,35005,Mujeres,F,De 35 a 44 años,Y35T44,Personas desempleadas registradas,UNE_REG,13
1,2021 Cuarto trimestre,2021-Q4,Artenara,35005,Mujeres,F,16 años o más,Y_GE16,Personas desempleadas registradas,UNE_REG,47
2,2021 Cuarto trimestre,2021-Q4,Artenara,35005,Mujeres,F,De 16 a 24 años,Y16T24,Personas desempleadas registradas,UNE_REG,5
3,2021 Cuarto trimestre,2021-Q4,Artenara,35005,Mujeres,F,55 años o más,Y_GE55,Personas desempleadas registradas,UNE_REG,15
4,2021 Cuarto trimestre,2021-Q4,Artenara,35005,Mujeres,F,De 25 a 34 años,Y25T34,Personas desempleadas registradas,UNE_REG,5
...,...,...,...,...,...,...,...,...,...,...,...
62197,2019 Primer trimestre,2019-Q1,Agulo,38002,Hombres,M,16 años o más,Y_GE16,Personas activas registradas,ACT_REG,262
62198,2019 Primer trimestre,2019-Q1,Agulo,38002,Hombres,M,De 16 a 24 años,Y16T24,Personas activas registradas,ACT_REG,13
62199,2019 Primer trimestre,2019-Q1,Agulo,38002,Hombres,M,55 años o más,Y_GE55,Personas activas registradas,ACT_REG,66
62200,2019 Primer trimestre,2019-Q1,Agulo,38002,Hombres,M,De 25 a 34 años,Y25T34,Personas activas registradas,ACT_REG,48


### `df_p_ocup_actividad`

In [22]:
df_p_ocup = pd.read_csv('../data/empleo/datasets-ISTAC_eparegOcupadaActividad-1921.csv')

# Eliminamos las columnas sobrantes
df_p_ocup.drop(
    columns = [
        'ESTADO_OBSERVACION#es',
        'CONFIDENCIALIDAD_OBSERVACION#es',
        'NOTAS_OBSERVACION#es'
    ],
    inplace = True
)

# Eliminamos las entradas con SEXO_CODE == _T (Total)
df_p_ocup.drop(df_p_ocup[df_p_ocup['SEXO_CODE'] == '_T'].index, inplace = True)

# Eliminamos las entradas de cada isla
rows_islas = df_p_ocup[df_p_ocup['TERRITORIO_CODE'].str.contains(r'ES\d{3}')].index
df_p_ocup.drop(
    rows_islas,
    inplace = True
)

# Renombramos la columna 'OBS_VALUE' a 'FREQUENCY'
df_p_ocup.rename(
    columns = {'OBS_VALUE': 'FREQUENCY'},
    inplace = True
)

display(df_p_ocup)

,TIME_PERIOD#es,TIME_PERIOD_CODE,SEXO#es,SEXO_CODE,TERRITORIO#es,TERRITORIO_CODE,ACTIVIDAD_ECONOMICA#es,ACTIVIDAD_ECONOMICA_CODE,FREQUENCY
0,2021 Cuarto trimestre,2021-Q4,Mujeres,F,Artenara,35005,Servicios,A4_04,156
1,2021 Cuarto trimestre,2021-Q4,Hombres,M,Artenara,35005,Servicios,A4_04,201
3,2021 Tercer trimestre,2021-Q3,Mujeres,F,Artenara,35005,Servicios,A4_04,152
4,2021 Tercer trimestre,2021-Q3,Hombres,M,Artenara,35005,Servicios,A4_04,198
6,2021 Segundo trimestre,2021-Q2,Mujeres,F,Artenara,35005,Servicios,A4_04,146
...,...,...,...,...,...,...,...,...,...
20728,2019 Tercer trimestre,2019-Q3,Hombres,M,Agulo,38002,Total,_T,185
20730,2019 Segundo trimestre,2019-Q2,Mujeres,F,Agulo,38002,Total,_T,163
20731,2019 Segundo trimestre,2019-Q2,Hombres,M,Agulo,38002,Total,_T,217
20733,2019 Primer trimestre,2019-Q1,Mujeres,F,Agulo,38002,Total,_T,173


### `df_p_parados`

In [23]:
df_p_parados = pd.read_csv('../data/empleo/datasets-ISTAC_eparegParadosActividad-1921.csv')

# Eliminamos las columnas sobrantes
df_p_parados.drop(
    columns = [
        'ESTADO_OBSERVACION#es',
        'CONFIDENCIALIDAD_OBSERVACION#es',
        'NOTAS_OBSERVACION#es'
    ],
    inplace = True
)

# Eliminamos las entradas con SEXO_CODE == _T (Total)
df_p_parados.drop(df_p_parados[df_p_parados['SEXO_CODE'] == '_T'].index, inplace = True)

# Eliminamos las entradas de cada isla
rows_islas = df_p_parados[df_p_parados['TERRITORIO_CODE'].str.contains(r'ES\d{3}')].index
df_p_parados.drop(
    rows_islas,
    inplace = True
)

# Renombramos la columna 'OBS_VALUE' a 'FREQUENCY'
df_p_parados.rename(
    columns = {'OBS_VALUE': 'FREQUENCY'},
    inplace = True
)

display(df_p_parados)

,TIME_PERIOD#es,TIME_PERIOD_CODE,SEXO#es,SEXO_CODE,TERRITORIO#es,TERRITORIO_CODE,ACTIVIDAD_ECONOMICA#es,ACTIVIDAD_ECONOMICA_CODE,FREQUENCY
0,2021 Cuarto trimestre,2021-Q4,Hombres,M,Arrecife,35004,Servicios,A4_04,1691
1,2021 Cuarto trimestre,2021-Q4,Mujeres,F,Arrecife,35004,Servicios,A4_04,2837
2,2021 Tercer trimestre,2021-Q3,Hombres,M,Arrecife,35004,Servicios,A4_04,1983
3,2021 Tercer trimestre,2021-Q3,Mujeres,F,Arrecife,35004,Servicios,A4_04,3294
4,2021 Segundo trimestre,2021-Q2,Hombres,M,Arrecife,35004,Servicios,A4_04,3275
...,...,...,...,...,...,...,...,...,...
12667,2019 Tercer trimestre,2019-Q3,Mujeres,F,El Pinar de El Hierro,38901,Total,_T,79
12668,2019 Segundo trimestre,2019-Q2,Hombres,M,El Pinar de El Hierro,38901,Total,_T,74
12669,2019 Segundo trimestre,2019-Q2,Mujeres,F,El Pinar de El Hierro,38901,Total,_T,85
12670,2019 Primer trimestre,2019-Q1,Hombres,M,El Pinar de El Hierro,38901,Total,_T,76


### `df_p_tcontrato`

In [24]:
df_p_tcontrato = pd.read_csv('../data/empleo/datasets-ISTAC_eparegParadosActividad-1921.csv')

# Eliminamos las columnas sobrantes
df_p_tcontrato.drop(
    columns = [
        'ESTADO_OBSERVACION#es',
        'CONFIDENCIALIDAD_OBSERVACION#es',
        'NOTAS_OBSERVACION#es'
    ],
    inplace = True
)

# Eliminamos las entradas con SEXO_CODE == _T (Total)
df_p_tcontrato.drop(df_p_tcontrato[df_p_tcontrato['SEXO_CODE'] == '_T'].index, inplace = True)

# Eliminamos las entradas de cada isla
rows_islas = df_p_tcontrato[df_p_tcontrato['TERRITORIO_CODE'].str.contains(r'ES\d{3}')].index
df_p_tcontrato.drop(
    rows_islas,
    inplace = True
)

# Renombramos la columna 'OBS_VALUE' a 'FREQUENCY'
df_p_tcontrato.rename(
    columns = {'OBS_VALUE': 'FREQUENCY'},
    inplace = True
)

display(df_p_tcontrato)

,TIME_PERIOD#es,TIME_PERIOD_CODE,SEXO#es,SEXO_CODE,TERRITORIO#es,TERRITORIO_CODE,ACTIVIDAD_ECONOMICA#es,ACTIVIDAD_ECONOMICA_CODE,FREQUENCY
0,2021 Cuarto trimestre,2021-Q4,Hombres,M,Arrecife,35004,Servicios,A4_04,1691
1,2021 Cuarto trimestre,2021-Q4,Mujeres,F,Arrecife,35004,Servicios,A4_04,2837
2,2021 Tercer trimestre,2021-Q3,Hombres,M,Arrecife,35004,Servicios,A4_04,1983
3,2021 Tercer trimestre,2021-Q3,Mujeres,F,Arrecife,35004,Servicios,A4_04,3294
4,2021 Segundo trimestre,2021-Q2,Hombres,M,Arrecife,35004,Servicios,A4_04,3275
...,...,...,...,...,...,...,...,...,...
12667,2019 Tercer trimestre,2019-Q3,Mujeres,F,El Pinar de El Hierro,38901,Total,_T,79
12668,2019 Segundo trimestre,2019-Q2,Hombres,M,El Pinar de El Hierro,38901,Total,_T,74
12669,2019 Segundo trimestre,2019-Q2,Mujeres,F,El Pinar de El Hierro,38901,Total,_T,85
12670,2019 Primer trimestre,2019-Q1,Hombres,M,El Pinar de El Hierro,38901,Total,_T,76


### `df_defun`

In [32]:
df_defun = pd.read_csv('../data/defunciones/DefCanariaStatTasa.csv').drop(columns = 'Unnamed: 0')

# Rellenar los registros de puntos con 0
df_defun['Defunciones'].replace(
    to_replace = r'^\.+$',
    value = 0,
    regex = True,
    inplace = True
)

# Eliminar los posibles puntos de centena intermedios
df_defun['Defunciones'] = df_defun['Defunciones'].str.replace('.','')

# Sustituye las comas decimales por puntos decimales
df_defun['Defunciones'] = df_defun['Defunciones'].str.replace(',','.')

# Convertir el tipo de dato a numérico
df_defun['Defunciones'] = df_defun['Defunciones'].astype(float)

# Eliminar los espacios sobrantes en los municipios
df_defun['Municipio'] = df_defun['Municipio'].str.strip()

# Corregir los nombres mal ordenados: 'nombre, (pronombre)'
def fix_municipality_name(mun_name):
    match_result = re.match(
        pattern = r'(.*) \((.*)\)',
        string = mun_name
    )
    if match_result:
        name, pronoun = match_result.groups()
        return f'{pronoun} {name}'
    else:
        return mun_name
    
df_defun['Municipio'] = df_defun['Municipio'].apply(fix_municipality_name)

# Corregir los nombres mal escritos de los municipios
df_defun['Municipio'].replace(
    to_replace = 'La Laguna' ,
    value = 'San Cristóbal de La Laguna',
    inplace = True
)
df_defun['Municipio'].replace(
    to_replace  = df_defun['Municipio'].sort_values().unique(),
    value = df_p_tcontrato['TERRITORIO#es'].sort_values().unique(),
    inplace = True
)

# Añadir los códigos de municipio
df_defun = pd.merge(
    left = df_defun,
    right = df_p_activa[['TERRITORIO#es', 'TERRITORIO_CODE']],
    how = 'left',
    left_on = 'Municipio',
    right_on = 'TERRITORIO#es'
)
df_defun.drop(columns = 'TERRITORIO#es', inplace = True)

display(df_defun)

,Municipio,Año,Sexo,Tipo,Defunciones,TERRITORIO_CODE
0,Arrecife,2021,Hombres,I. CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITA...,37.28,35004
1,Arrecife,2021,Hombres,I. CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITA...,37.28,35004
2,Arrecife,2021,Hombres,I. CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITA...,37.28,35004
3,Arrecife,2021,Hombres,I. CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITA...,37.28,35004
4,Arrecife,2021,Hombres,I. CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITA...,37.28,35004
...,...,...,...,...,...,...
4111771,Vilaflor de Chasna,2019,Mujeres,XX. CAUSAS EXTERNAS DE MORBILIDAD Y DE MORTALIDAD,0.00,38052
4111772,Vilaflor de Chasna,2019,Mujeres,XX. CAUSAS EXTERNAS DE MORBILIDAD Y DE MORTALIDAD,0.00,38052
4111773,Vilaflor de Chasna,2019,Mujeres,XX. CAUSAS EXTERNAS DE MORBILIDAD Y DE MORTALIDAD,0.00,38052
4111774,Vilaflor de Chasna,2019,Mujeres,XX. CAUSAS EXTERNAS DE MORBILIDAD Y DE MORTALIDAD,0.00,38052


### `df_pob`

In [ ]:
df_pob_2019: pd.DataFrame = gpd.read_file('../data/poblacion/Municipios-20190101.json')

# Seleccionamos unicamente las columnas de interes
df_pob_2019 = df_pob_2019.filter([
    'ano',
    'municipio',
    'poblacion',
    'poblacion_hombres',
    'poblacion_mujeres',
    'poblacion_edad_media',
    'geometry'
])

display(df_pob_2019)

In [ ]:
df_pob_2020: pd.DataFrame = gpd.read_file('../data/poblacion/Municipios-20200101.json')

# Seleccionamos unicamente las columnas de interes
df_pob_2020 = df_pob_2020.filter([
    'ano',
    'municipio',
    'poblacion',
    'poblacion_hombres',
    'poblacion_mujeres',
    'poblacion_edad_media',
    'geometry'
])

display(df_pob_2020)

In [ ]:
df_pob_2021: pd.DataFrame = gpd.read_file('../data/poblacion/Municipios-20210101.json')

# Seleccionamos unicamente las columnas de interes
df_pob_2021 = df_pob_2021.filter([
    'ano',
    'municipio',
    'poblacion',
    'poblacion_hombres',
    'poblacion_mujeres',
    'poblacion_edad_media',
    'geometry'
])

display(df_pob_2021)

In [ ]:
df_pob = pd.concat([
    df_pob_2019,
    df_pob_2020,
    df_pob_2021
])

display(df_pob)

## Historia